In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../lib'))
from c19us_nyt import df_us as df_nyt
from c19us_jhu import df_us as df_jhu
import numpy as np

In [2]:
df_nyt = df_nyt.reset_index()[['date','fips','day','county','state','cases','deaths']]
print(df_nyt.shape)
df_nyt.head()

(134025, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [3]:
df_nyt = df_nyt.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_nyt.shape)
df_nyt.head()

(134025, 7)


,date,fips,day,county,state,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [4]:
df_jhu = df_jhu.reset_index()[['date','fips','day','county','state','cases','deaths','recovered','active']]
print(df_jhu.shape)
df_jhu.head()

(141333, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,36061,0,New York City,New York,9654,63,0,0
1,2020-03-22,36059,0,Nassau,New York,1900,4,0,0
2,2020-03-22,36119,0,Westchester,New York,1873,0,0,0
3,2020-03-22,36103,0,Suffolk,New York,1034,9,0,0
4,2020-03-22,36087,0,Rockland,New York,455,1,0,0


In [5]:
df_jhu = df_jhu.groupby(['date','fips','day','county','state']).sum().reset_index()
print(df_jhu.shape)
df_jhu.head()

(141318, 9)


,date,fips,day,county,state,cases,deaths,recovered,active
0,2020-03-22,1001,0,Autauga,Alabama,0,0,0,0
1,2020-03-22,1003,0,Baldwin,Alabama,2,0,0,0
2,2020-03-22,1005,0,Barbour,Alabama,0,0,0,0
3,2020-03-22,1007,0,Bibb,Alabama,0,0,0,0
4,2020-03-22,1009,0,Blount,Alabama,0,0,0,0


In [6]:
states = pd.read_csv('../../covid-19/data/state_codes.csv')
states.head()

,Code,State
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [7]:
df = pd.merge(df_nyt, df_jhu, on=['date','fips','day','county','state'], how='outer')\
    .sort_values('date')
print(df.shape)
df.head()

(275343, 11)


,date,fips,day,county,state,cases_x,deaths_x,cases_y,deaths_y,recovered,active
0,2020-01-21,53061,0,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
1,2020-01-22,53061,1,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
2,2020-01-23,53061,2,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN
3,2020-01-24,17031,3,Cook,Illinois,1.0,0.0,NaN,NaN,NaN,NaN
4,2020-01-24,53061,3,Snohomish,Washington,1.0,0.0,NaN,NaN,NaN,NaN


In [8]:
df.loc[df.county=='New York City','cases_x'] = df.loc[df.county=='New York City','cases_y']

In [9]:
df = df[df.cases_x.isna()==False]

In [10]:
df['cases'] = df['cases_y'].fillna(df['cases_x'])
df['deaths'] = df['deaths_y'].fillna(df['deaths_x'])
df = df.drop(['cases_x','cases_y','deaths_x','deaths_y'], axis=1)
df['recovered'] = df['recovered'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['active'] = df['active'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['cases'] = df['cases'].map(lambda x: np.nan if np.isnan(x) else int(x))
df['deaths'] = df['deaths'].map(lambda x: np.nan if np.isnan(x) else int(x))
print(df.shape)
df.head()

(134076, 9)


,date,fips,day,county,state,recovered,active,cases,deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [11]:
df = df.reset_index()[['date','fips','day','county','state','recovered','active','cases','deaths']]
df.columns = ['Date','FIPS','Day','County','State','Recovered','Active','Cases','Deaths']
print(df.shape)
df.head()

(134076, 9)


,Date,FIPS,Day,County,State,Recovered,Active,Cases,Deaths
0,2020-01-21,53061,0,Snohomish,Washington,NaN,NaN,1,0
1,2020-01-22,53061,1,Snohomish,Washington,NaN,NaN,1,0
2,2020-01-23,53061,2,Snohomish,Washington,NaN,NaN,1,0
3,2020-01-24,17031,3,Cook,Illinois,NaN,NaN,1,0
4,2020-01-24,53061,3,Snohomish,Washington,NaN,NaN,1,0


In [12]:
df_ = pd.merge(df,states,on='State',how='left')

In [13]:
# indexNames = df_[df_['County'].isin(['Bronx','Queens','Manhattan','Staten Island','Brooklyn'])].index
# df_.drop(indexNames , inplace=True)

In [14]:
all_counties = pd.read_csv('../../covid-19/data/county_fips.csv')
all_counties['County'] = all_counties.County.map(lambda x: x.replace('County','').strip())
all_counties.head()

,FIPS,County,Code
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga,AL
3,1003,Baldwin,AL
4,1005,Barbour,AL


In [15]:
all_counties.loc[all_counties['County']=='Queens',:]

,FIPS,County,Code
1902,36081,Queens,NY


In [16]:
all_counties.loc[all_counties['County']=='Bronx',:]

,FIPS,County,Code
1864,36005,Bronx,NY


In [17]:
all_counties.loc[all_counties['County']=='Kings',:]

,FIPS,County,Code
207,6031,Kings,CA
1885,36047,Kings,NY


In [18]:
all_counties.loc[all_counties['County']=='Richmond',:]

,FIPS,County,Code
519,13245,Richmond,GA
1904,36085,Richmond,NY
2001,37153,Richmond,NC
2943,51159,Richmond,VA


In [19]:
all_counties[all_counties['FIPS']==36061]

,FIPS,County,Code
1892,36061,New York,NY


In [20]:
all_counties.loc[all_counties['FIPS']==36061,'County'] = 'Manhattan'

In [32]:
NY_Burrows = ['Manhattan','Queens','Kings','Bronx','Richmond']
NY_Burrow_fips = [36061,36081,36047,36005,36085]

all_dates = pd.DataFrame(columns=['Date','FIPS','Recovered','Active','Cases','Deaths','County','Code'])
for d in sorted(set(df_nyt['date'])):
    print('Testing... ' + f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv')
    dff = df_[df_['Date'] == d]
    if 'New York City' in dff['County'].tolist():
        for c, f in zip(NY_Burrows, NY_Burrow_fips):
            new_row = dff.copy()[dff['County']=='New York City'].reset_index(drop=True)
            new_row.loc[0,['FIPS',
                           'County',
                           'Recovered',
                           'Active',
                           'Cases',
                           'Deaths']] = [f,
                                          c, 
                                          new_row.loc[0,'Recovered']/5,
                                          new_row.loc[0,'Active']/5,
                                          new_row.loc[0,'Cases']/5,
                                          new_row.loc[0,'Deaths']/5
                                         ]
            dff = dff.append(new_row, ignore_index=True)
        dff = dff.loc[np.where(dff['County']!='New York City')]        
    added_counties = pd.merge(dff, all_counties, how='right', on='FIPS')
    added_counties = added_counties.drop(['County_x','Code_x'], axis=1)
    added_counties.columns = ['Date','FIPS','Day','State','Recovered','Active','Cases','Deaths','County','Code']
    added_counties['Cases'] = added_counties['Cases'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['Deaths'] = added_counties['Deaths'].map(lambda x: 0 if np.isnan(x) else int(x))
    added_counties['FIPS'] = added_counties['FIPS'].map(lambda x: '0' + str(x) if (int(x) <= 9999) else str(x))
    added_counties = pd.merge(added_counties, states, how='left', on='Code')
    added_counties['State'] = added_counties['State_y'].values
    added_counties = added_counties.drop(['State_x','State_y'], axis=1)
    
    all_dates = all_dates.append(added_counties, ignore_index=True)

    added_counties.to_csv(f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv', index=False)
all_dates = all_dates[all_dates['Date'].isna()==False]
all_dates.to_csv(f'../../covid-19/data/daily_cases_USA_counties.csv', index=False)

Testing... ../../covid-19/data/county_data/date_2020-01-21.csv
Testing... ../../covid-19/data/county_data/date_2020-01-22.csv
Testing... ../../covid-19/data/county_data/date_2020-01-23.csv
Testing... ../../covid-19/data/county_data/date_2020-01-24.csv
Testing... ../../covid-19/data/county_data/date_2020-01-25.csv
Testing... ../../covid-19/data/county_data/date_2020-01-26.csv
Testing... ../../covid-19/data/county_data/date_2020-01-27.csv
Testing... ../../covid-19/data/county_data/date_2020-01-28.csv
Testing... ../../covid-19/data/county_data/date_2020-01-29.csv
Testing... ../../covid-19/data/county_data/date_2020-01-30.csv
Testing... ../../covid-19/data/county_data/date_2020-01-31.csv
Testing... ../../covid-19/data/county_data/date_2020-02-01.csv
Testing... ../../covid-19/data/county_data/date_2020-02-02.csv
Testing... ../../covid-19/data/county_data/date_2020-02-03.csv
Testing... ../../covid-19/data/county_data/date_2020-02-04.csv
Testing... ../../covid-19/data/county_data/date_2020-02

In [33]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').head()

(134246, 10)


,Date,FIPS,Recovered,Active,Cases,Deaths,County,Code,Day,State
0,2020-01-21,53061,NaN,NaN,1,0,Snohomish,WA,0.0,Washington
1,2020-01-22,53061,NaN,NaN,1,0,Snohomish,WA,1.0,Washington
2,2020-01-23,53061,NaN,NaN,1,0,Snohomish,WA,2.0,Washington
3,2020-01-24,17031,NaN,NaN,1,0,Cook,IL,3.0,Illinois
4,2020-01-24,53061,NaN,NaN,1,0,Snohomish,WA,3.0,Washington


In [34]:
# all_dates[((all_dates['Date'] == '2020-04-26')) &\
#          (all_dates['Code'] == 'NY')] \
#         .sort_values('Cases', ascending=False)[['County','Cases','Deaths']]

In [35]:
df2 = df[['Date','Day','State','Recovered','Active','Cases','Deaths']]\
    .groupby(['Date','State'])\
    .sum()\
    .reset_index() #.drop('Day', axis=1)

In [36]:
df3 = pd.merge(df2, states, how='left', on='State')
df3['Recovered'] = df3['Recovered'].map(lambda x: int(x))
df3['Active'] = df3['Active'].map(lambda x: int(x))
df3 = df3[df3['Date'].isin(df_nyt['date'])]

In [37]:
print(df3.shape)
df3.head()

(3615, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [38]:
df3.to_csv('../../covid-19/data/daily_cases_USA_states.csv', index=False)

In [39]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').head()

(3615, 8)


,Date,State,Day,Recovered,Active,Cases,Deaths,Code
0,2020-01-21,Washington,0,0,0,1,0,WA
1,2020-01-22,Washington,1,0,0,1,0,WA
2,2020-01-23,Washington,2,0,0,1,0,WA
3,2020-01-24,Illinois,3,0,0,1,0,IL
4,2020-01-24,Washington,3,0,0,1,0,WA


In [40]:
all_dates.Date.min(), all_dates.Date.max()

(Timestamp('2020-01-21 00:00:00'), Timestamp('2020-05-11 00:00:00'))